# Air Traffic Input Data Epxloration and Graph Construction for Air Traffic Prediction

This notebook helps readers get better understanding of the input data format of the synthetic air transportation graph, and how it meets GraphStorm's graph construction command requirements. Readers should run the `Synthetic_Airport_Traffic_wAirlines.ipynb` first to generate the three parquet files that will be used in this notebook.

GraphStorm can handel experise-level graph data, measured in billions of nodes and tens of billions of edges. This synthetic air trasportation, however, is relatively small. Therefore we will use the `graphstorm.gconstruct.construct_graph` command that can run on a single machine. Details of the input data format can be found in GraphStorm's [Input Raw Data Specification](https://graphstorm.readthedocs.io/en/latest/cli/graph-construction/raw_data.html) documentations. For large graph data that might consume more memory in multiple machines, readers can refer to GraphStorm's [Distributed Graph Construction
](https://graphstorm.readthedocs.io/en/latest/cli/graph-construction/distributed/index.html) documentations.

In [1]:
import os
import json
import pandas as pd

## Explore the Input Data

In order to deal with time series data, we store them as lists in one column. In such way, we store the time series data as a $N \times T$ tensor, where $N$ is the number of samples, and $T$ is the number of days. Then we can use this tensor for modeling.

In [2]:
airport_path = './airport.parquet'
airport_node_df = pd.read_parquet(airport_path)

In [3]:
airport_node_df.sample(7)

,iata_code,latitude_deg,longitude_deg,inventory_amounts
282,THR,35.689201,51.313400,"[26.3705, 113.2035, 235.0796, 294.976, 305.833..."
176,TIA,41.414700,19.720600,"[27.3904, 125.6269, 256.6235, 315.6315, 327.67..."
336,BPS,-16.438426,-39.080584,"[33.1111, 124.6781, 247.1536, 303.8346, 315.60..."
6,YQB,46.791100,-71.393303,"[30.869, 140.0717, 262.2105, 318.3473, 330.663..."
219,MLA,35.857498,14.477500,"[35.7183, 137.6625, 265.6365, 320.0791, 330.87..."
37,LHR,51.470600,-0.461941,"[25.2028, 114.7148, 240.5703, 293.9926, 302.89..."
192,MRS,43.438088,5.212500,"[38.9391, 136.8784, 255.9467, 329.7522, 346.73..."


In [4]:
demand_edge_path = './demand_edge.parquet'
demand_edge_df = pd.read_parquet(demand_edge_path)

In [5]:
demand_edge_df.sample(7)

,src_code,dst_code,demands
157962,PTP,HRI,"[0.0, 1.0, 0.76, 1.0, 0.37, 0.0, 0.0, 0.0, 0.0..."
148213,IBZ,VVO,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
55890,TLL,MKE,"[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
108989,DUR,CHC,"[0.0, 0.0, 0.76, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
25923,GSV,GBE,"[0.15, 0.24, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
59604,PVD,PBI,"[0.86, 0.2, 1.24, 0.0, 0.0, 0.0, 0.0, 0.12, 2...."
3279,SID,YVR,"[0.0, 0.66, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."


In [6]:
traffic_edge_path = 'traffic_edge.parquet'
traffic_edge_df = pd.read_parquet(traffic_edge_path)

In [7]:
traffic_edge_df.sample(7)

,src_code,dst_code,capacity,traffics
4300,OMA,CMH,3.3125,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1297,DEN,ANC,3.0000,"[0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.2..."
2102,GYD,DAC,0.5000,"[0.0238, 0.0238, 0.0238, 0.0238, 0.0238, 0.023..."
2614,JAX,CLT,2.0000,"[0.0129, 0.0129, 0.0129, 0.0129, 0.0129, 0.012..."
5766,SLC,LGA,3.0000,"[0.0224, 0.0224, 0.0224, 0.0224, 0.0224, 0.022..."
795,BUF,RDU,1.5000,"[0.015, 0.015, 0.015, 0.015, 0.015, 0.015, 0.0..."
5051,RDU,ORF,2.5000,"[0.0213, 0.0213, 0.0213, 0.0213, 0.0213, 0.021..."


### Prepare the JSON file for `graphstorm.gconstruct.construct_graph` command

The `graphstorm.gconstruct.construct_graph` command replys on a JSON file to understand the given graph data. So here we provide the JSON file of the synthetic air transportation network. For more details of each field of the JSON file and format requirements, readers can refer to the [Configuration JSON Object Explanations](https://graphstorm.readthedocs.io/en/latest/cli/graph-construction/single-machine-gconstruct.html#configuration-json-object-explanations).

In [8]:
air_traffic_json = {"version": "gconstruct-v0.1"}

Node objects record the node types in a graph, in where node data table path, and node features or labels if have, are presented. It is worth noting that we normalize some features by using GraphStorm's built-in transform (feature engineering) functions. This operation could help GNN models to converge faster.

In [9]:
nodes = []
airport = {
    "node_type": "airport",
    "format": {
        "name": "parquet"
    },
    "files": [
        airport_path
    ],
    "node_id_col": "iata_code",
    "features": [
        {
            "feature_col": "latitude_deg",
            "feature_name": "latitude",
            "transform": {"name": "max_min_norm",
                          "max_val": 90.,
                          "min_val": -90.}
        },
        {
            "feature_col": "longitude_deg",
            "feature_name": "longitude",
            "transform": {"name": "max_min_norm",
                          "max_val": 180.,
                          "min_val": -180.}
        },
        {
            "feature_col": "inventory_amounts",
            "feature_name": "inventory_amounts",
            "transform": {"name": "max_min_norm",
                          "max_val": 1000.,
                          "min_val": 0.}
        }
    ],
    "labels": [
        {
            "label_col": "inventory_amounts",
            "task_type": "regression",
            "split_pct": [
                0.8,
                0.1,
                0.1
            ]
        }
    ]
}

nodes.append(airport)

Edge objects are similar as node objects except that they have a "relation" field to record source node type, edge type, and destination type in a list.

In [10]:
edges = []
ap_demand_ap = {
    "relation": [
        "airport",
        "demand",
        "airport"
    ],
    "format": {
        "name": "parquet"
    },
    "files": [
        demand_edge_path
    ],
    "source_id_col": "src_code",
    "dest_id_col": "dst_code",
    "features": [
        {
            "feature_col": "demands",
            "feature_name": "demands"
        }
    ]
}
ap_traffic_ap = {
    "relation": [
        "airport",
        "traffic",
        "airport"
    ],
    "format": {
        "name": "parquet"
    },
    "files": [
        traffic_edge_path
    ],
    "source_id_col": "src_code",
    "dest_id_col": "dst_code",
    "features": [
        {
            "feature_col": "capacity",
            "feature_name": "capacity"
        },
        {
            "feature_col": "traffics",
            "feature_name": "traffics"
        }
    ]
}
edges.append(ap_demand_ap)
edges.append(ap_traffic_ap)

In [11]:
air_traffic_json['nodes'] = nodes
air_traffic_json['edges'] = edges

In [12]:
# Save to a local file, named config.json
with open(os.path.join("config.json"), "w") as f:
    json.dump(air_traffic_json, f, indent=4)

## Run GraphStorm `gconstruct` Command to Process Air Traffic Data for Using GraphStorm

With the tree parquet files and the `config.json` JSON file, we then run graph construction.

First, let's install GraphStorm and its depenencies, assuming on a CPU machine.

In [13]:
!pip install graphstorm

# If using CPU instances
!pip install torch==2.1.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!pip install dgl==1.1.3 -f https://data.dgl.ai/wheels-internal/repo.html

Looking in indexes: https://download.pytorch.org/whl/cpu
Looking in links: https://data.dgl.ai/wheels-internal/repo.html


Now, run the `gconstruct.construct_graph` command with a few arguments. For example, as the air traffic graph data is relatively small, we only split the data into one partition, i.e., `--num-part 1`. And the output data will be saved to a folder named `gs_1p` to be used in the model training and inference at the `AirTrafficPrediction.ipynb` notebook.

In [14]:
!python -m graphstorm.gconstruct.construct_graph \
           --conf-file config.json \
           --output-dir gs_1p/ \
           --num-parts 1 \
           --graph-name air_traffic

INFO:root:The graph has 1 node types and 2 edge types.
INFO:root:Node type airport has 400 nodes
INFO:root:Edge type ('airport', 'demand', 'airport') has 159600 edges
INFO:root:Edge type ('airport', 'traffic', 'airport') has 6628 edges
INFO:root:Node type airport has features: ['latitude', 'longitude', 'inventory_amounts', 'train_mask', 'val_mask', 'test_mask'].
INFO:root:Train/val/test on airport with mask train_mask, val_mask, test_mask: 320, 40, 40
INFO:root:Note: Custom train, validate, test mask information for nodes are not collected.
INFO:root:Edge type ('airport', 'demand', 'airport') has features: ['demands'].
INFO:root:Edge type ('airport', 'traffic', 'airport') has features: ['capacity', 'traffics'].
The graph has 1 node types and balance among 3 types
Converting to homogeneous graph takes 0.006s, peak mem: 13.504 GB
Save partitions: 0.008 seconds, peak memory: 14.958 GB
There are 166228 edges in the graph and 0 edge cuts for 1 partitions.
INFO:root:Graph construction genera